In [1]:
import mne
from pathlib import Path

from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci

path = Path(r"C:\Users\erik\IES_codebase\pythonProjects\EEG_Exoskeleton\data\raw_data")

In [2]:
raw_fnames = []

subjects = [1, 2]
runs = [6, 10, 14]

for sub in subjects:
    fname = eegbci.load_data(subject=sub, runs=runs, path=path)
    raw_fnames.extend(fname)

raws = [read_raw_edf(f, preload=True) for f in raw_fnames]
raw = concatenate_raws(raws)

raw.rename_channels(lambda x: x.strip("."))  # remove dots from channel names
# rename descriptions to be more easily interpretable
raw.annotations.rename(dict(T1="hands", T2="feet"))

montage = mne.channels.make_standard_montage("standard_1005")

mne.datasets.eegbci.standardize(raw)  # set channel names
raw.set_montage(montage, verbose=False)

Extracting EDF parameters from C:\Users\erik\IES_codebase\pythonProjects\EEG_Exoskeleton\data\raw_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\erik\IES_codebase\pythonProjects\EEG_Exoskeleton\data\raw_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\erik\IES_codebase\pythonProjects\EEG_Exoskeleton\data\raw_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\erik\IES_codebase\pythonProjects\EEG_Exoskeleton\data\raw_data\

<RawEDF | S001R06.edf, 64 x 119040 (744.0 s), ~58.2 MB, data loaded>

In [3]:
tmin, tmax = -1, 4
event_ids = dict(hands=2, feet=3)  # map event IDs to tasks

In [4]:
epochs = mne.Epochs(
    raw,
    event_id=["hands", "feet"],
    tmin=tmin - 0.5,
    tmax=tmax + 0.5,
    baseline=None,
    preload=True,
)

Used Annotations descriptions: ['T0', 'feet', 'hands']
Ignoring annotation durations and creating fixed-duration epochs around annotation onsets.
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 90 events and 961 original time points ...
3 bad epochs dropped


In [5]:
epochs_train = epochs.copy().crop(tmin=1.0, tmax=2.0)
labels = epochs.events[:, -1] - 2

In [6]:
epochs_data = epochs.get_data(copy=False)
epochs_data_train = epochs_train.get_data(copy=False)

In [7]:
from sklearn.model_selection import ShuffleSplit, cross_val_score

cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP

lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

In [9]:
from sklearn.pipeline import Pipeline

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([("CSP", csp), ("LDA", lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None)

Computing rank from data with rank=None
    Using tolerance 0.00038 (2.2e-16 eps * 64 dim * 2.7e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00042 (2.2e-16 eps * 64 dim * 2.9e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00042 (2.2e-16 eps * 64 dim * 3e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIR

In [10]:
import numpy as np

class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1.0 - class_balance)
print(f"Classification accuracy: {np.mean(scores)} / Chance level: {class_balance}")

Classification accuracy: 0.3388888888888889 / Chance level: 0.5172413793103449
